In [5]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import PyPDF2
import os


#### Collect the links on the page

In [8]:
url = "https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/"
response = requests.get(url + "/?showSvar=true&term=&isFilterOpen=true&page=2")

html_doc = response.text

soup = bs(html_doc, 'html.parser')

# Find the ul class that contains the articles
articles = soup.find("ul", class_="link-list")

# Create a dictionary to store the sites and pdfs
sites = {}
pdfs = {}

# Iterate through every li tag in the ul class
for article in articles.find_all("li"):
    if article.text.find("PDF") != -1:
        # Removes the (PDF) from the name of the actor (the writer of the article)
        actor = article.text.split("(")[0][:-1]
        pdfs[actor] = "https://www.regjeringen.no/" + article.find("a")["href"]
        continue
    sites[article.text] = url + article.find("a")["href"]


print(len(sites))
print(len(pdfs))


ConnectionError: HTTPSConnectionPool(host='www.regjeringen.no', port=443): Max retries exceeded with url: /no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052//?showSvar=true&term=&isFilterOpen=true&page=2 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023696914AA0>: Failed to resolve 'www.regjeringen.no' ([Errno 11001] getaddrinfo failed)"))

#### Download pdfs

In [1]:
import PyPDF2
import os
# Make a directory to store the pdfs
output_dir = "../../dataset/pdf/txt"

os.makedirs(output_dir, exist_ok=True)

# Iterate through the pdfs and download them
for actor, pdf in pdfs.items():
    response = requests.get(pdf)
    output_file = os.path.join(output_dir, actor + ".pdf")
    with open(output_file, "wb") as f:
        f.write(response.content)


NameError: name 'pdfs' is not defined

In [3]:

# Create a directory to store the text files
output_dir = "../../dataset/txt"
os.makedirs(output_dir, exist_ok=True)

#Read pdfs

pdf_dir = "../../dataset/pdf"



# Iterate through the PDFs and save them as text files
for actor, pdf in pdfs.items():
    # Open the PDF file
    with open(pdf, "rb") as f:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(f)
        
        # Extract the text from each page of the PDF
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        
        # Save the text as a text file
        output_file = os.path.join(output_dir, actor + ".txt")
        with open(output_file, "w") as f:
            f.write(text)
        
        

NameError: name 'pdfs' is not defined

#### Read the htmls and preprocess them

In [12]:
df_texts = pd.DataFrame(columns=['actor', 'paragraph', 'date'])

# Read the html files and extract the text

def preprocess_html(texts : list):
    # Remove the html tags
    paragraphs = [paragraph.text.strip() for paragraph in texts if paragraph.text != ""]

    # Remove the header "Høringssvar til NOU 2023: ...."
    paragraphs = [sections for sections in paragraphs if sections.find("Høringssvar til NOU 2023") == -1]

    return paragraphs




for actor, site in sites.items():
    # Remove the spaces at the end of the actor
    actor = actor.strip()
    print(actor.__repr__())
    response = requests.get(site)
    html_doc = response.text
    soup = bs(html_doc, 'html.parser')
    # Find the text of the article
    texts = soup.find("div", class_="article-body").find_all("p")
    text = preprocess_html(texts)

    # Find the date of the article
    date = soup.get_text().split("Dato")[1].split("\n")[0].split(" ")[1]


    for paragraph in text:
        df_texts.loc[len(df_texts)] = [actor, paragraph, date]
    


print(actor)
df_texts
df_texts.to_csv("../../dataset/html_hearings_page3.csv", index=False)


'Sivilingeniør Arve Gleissner Gustavsen'
'Skagerak Energi'
'Skattedirektoratet (uten merknader)'
'Skift - Næringslivets Klimaledere'
'Skognæringa Kyst SA'
'Småkraftforeninga'
'SoCentral AS'
'Sokkeldirektoratet'
'Sola INP'
'Spire'
'Standard Norge'
'Statens Arbeidsmiljøinstitutt'
'Statistisk sentralbyrå'
'Statkraft AS'
'Statsbygg'
'Statsforvaltaren i Vestland'
'Statsforvalteren i Vestfold og Telemark'
'Stavanger kommune'
'Steinar Eikemo'
'Stig Eidesmo'
'Stord Industri og Næringsparti (Stord INP)'
'Sunnfjord kommune'
'Syklistforeningen'
'Tekna - Teknisk-naturvitenskapelig forening'
'Telemark fylkeskommune'
'Thom'
'Treindustrien'
'Troms fylkeskommune'
'Tromsø kommune'
'Trondheim kommune'
'Trysil kommune'
'Tysvær Industri- og Næringspartiet'
'Tønsberg kommune'
'Unio'
'Universitetet i Stavanger'
'Universitets- og høgskolerådet'
'Vaksdal INP'
'Vardø kommune'
'Vennesla kommune'
'Vestlandsforsking'
'Vestnes Industri- og Næringsparti (INP)'
'Veterinærinstituttet'
'Veterinært bærekraftforum'
'Vit

In [13]:
df_texts

,actor,paragraph,date
0,Sivilingeniør Arve Gleissner Gustavsen,Høringsutvalgets rapporten fra 25. oktober 202...,03.02.2024
1,Sivilingeniør Arve Gleissner Gustavsen,Dette er nødvendig for å respondere på oppdrag...,03.02.2024
2,Sivilingeniør Arve Gleissner Gustavsen,Rapporten har fokus på utslippsreduksjon av CO...,03.02.2024
3,Sivilingeniør Arve Gleissner Gustavsen,Det er organisasjonen Klimapanelet som mener a...,03.02.2024
4,Sivilingeniør Arve Gleissner Gustavsen,Å gi en tilstrekkelig og god beskrivelse av de...,03.02.2024
...,...,...,...
635,ZERO,Zero støtter utvalgets syn på viktigheten av å...,23.02.2024
636,ZERO,Det finnes også andre måter å minimere eller e...,23.02.2024
637,ZERO,Samtidig er det viktig å både holde fast på kl...,23.02.2024
638,ZERO,Alle prosjekter som skal inn i NTP må kunne re...,23.02.2024
